In [1]:
!pip install kaggle

  Using cached https://files.pythonhosted.org/packages/a6/a5/c0b6468d3824fe3fde30dbb5e1f687b291608f9473681bbf7dabbf5a87d7/text_unidecode-1.3-py2.py3-none-any.whl
  Running setup.py bdist_wheel for python-slugify ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/67/b8/ba/041548f30a6fc058c9b3f79a5b7b6aea925a15dd1e5c4992a4
Successfully built python-slugify
You are using pip version 9.0.3, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install xgboost

    100% |████████████████████████████████| 737kB 1.1MB/s eta 0:00:01
  Running setup.py bdist_wheel for xgboost ... error
  Complete output from command /srv/app/venv/bin/python3.6 -u -c "import setuptools, tokenize;__file__='/tmp/pip-build-wxfxeqy_/xgboost/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /tmp/tmpj1jzuy6rpip-wheel- --python-tag cp36:
  running bdist_wheel
  running build
  running build_py
  INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
  INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt
  creating build
  creating build/lib.linux-x86_64-3.6
  creating build/lib.linux-x86_64-3.6/xgboost
  copying xgboost/sklearn.py -> build/lib.linux-x86_64-3.6/xgboost
  copying xgboost/dask.py -> build/lib.linux-x86_64-3.6/xgboost
  copying xgboost/callback.py -> build/lib.linux-x86_64-3.6/xgboost
  copying xgb

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datascience import *
import json
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
import xgboost as xgb

ModuleNotFoundError: No module named 'xgboost'

# Career Exploration Final Project: TMDB Box Office Prediction

### Table Of Contents

* [1. Exploratory Data Analysis](#eda)
* [2. Feature Engineering and Data Cleaning](#feature-engineering)
* [3. Modeling](#modeling)
    * [3.1 Validation and Evaluation](#validation)
    * [3.2 Linear Regression](#linear-regression)
    * [3.3 Regularized Regression](#reg)
    * [3.4 Random Forest](#random-forest)
    * [3.5 Neural Network](#nn)
    * [3.6 XGBoost](#xgb)


### Hosted by and maintained by the [Students Association of Applied Statistics (SAAS)](https://saas.berkeley.edu).  Authored by [Ajay Raj](mailto:araj@berkeley.edu).

For your final project in Career Exploration, you will be participating in a **Kaggle competition**, a data science and machine learning competition where you use *real* data and develop models to solve *real* problems.

## Description

The problem: given data about a movie (runtime, budget, cast, crew), predict the **overall worldwide box office revenue** it will make.

You'll be competing in [this Kaggle competition](https://www.kaggle.com/c/tmdb-box-office-prediction). Note that this competition has already completed, so you won't be competing against other Kagglers, but you'll be competing against your fellow CXers on a private leaderboard. For information on where the training data came from and how you're predictions are evaluated (turned into a score), check out the Kaggle competition link.

**Note:** There is not much guidance provided in this project (on purpose). You'll be doing a lot of going through [previous lectures](https://github.com/SUSA-org/Spring-2019-Career-Exploration/blob/master/CX-Final-Project/CX-Final-Project-Starter.ipynb) to try to adapt the code provided there to this dataset, and reading documentation that's been linked in most of the problems. We are pushing you, fledgling data scientists, out of the nest and letting you spread your wings and fly.

## Setup

1. Create a Kaggle account at kaggle.com
2. Go to the [Kaggle competition page](https://www.kaggle.com/c/tmdb-box-office-prediction) and click "Late Submission", and register for the competition/
3. Go to the 'Account' tab of your user profile (https://www.kaggle.com/YOUR-USERNAME/account) and select 'Create API Token'
4. Download the `kaggle.json` file, which contains a dictionary with your Kaggle credentials
5. Put them in the `KAGGLE_USER_DATA` variable

In [5]:
TEAM_NAME = "Popcorn loves movie"

In [6]:
KAGGLE_USER_DATA = {"username":"davidljy","key":"557501a37bb3e05db6e600e5fe2156be"}

## Data Loading

In [7]:
train = pd.read_csv('data/train.csv')

In [8]:
test = pd.read_csv('data/test.csv')

In [9]:
X_train, y_train = train.drop('revenue', axis=1), train['revenue']
X_test = test

When we do EDA and feature engineering on a dataset, we often examine the training points and the test points together, so when you do complex feature engineering and data cleaning, you don't need to do twice or worry about your transformations not applying to test set.

In [10]:
df = pd.concat((X_train, X_test), axis=0)

<span id="eda"></span>

## 1. Exploratory Data Analysis

Provide two plots that demonstrate interesting aspects of the dataset, and especially certain features' influence on the target variable, revenue.

Since you won't be "submitting" this notebook anywhere, this part of the project is technically optional, but it is a **crucial** part of the data science process, so we *highly* recommend you do this, because it will inform how you complete the next parts of the project.

In [11]:
#budget, popularity, runtime
X_train_numeric= X_train.select_dtypes([np.number])
X_train_numeric.head(5)

,id,budget,popularity,runtime
0,1,14000000,6.575393,93.0
1,2,40000000,8.248895,113.0
2,3,3300000,64.299990,105.0
3,4,1200000,3.174936,122.0
4,5,0,1.148070,118.0


In [12]:
# add the variable of belong to the column
df_boolean=df["belongs_to_collection"].isnull()
df['belong']=df_boolean
df.belong=df.belong.astype(int)
# add language to the column
df["language"]=(df["original_language"]=='en')
df.language=df.language.astype(int)

In [13]:
def proc_json(string, key):
    try:
        data = eval(string)
        return ",".join([d[key] for d in data])
    except:
        return ''
### Formatting the `genres` column
clean_genres = df.genres.apply(lambda x: proc_json(x, 'name'))
df["Converted_genres"]=clean_genres

In [14]:
#add dummy for Comedy
a=[]
for genres in df["Converted_genres"]:
    if "Comedy" in genres:
        a.append(1)
    else:
        a.append(0)
df["Comedy"]=a
#add dummy for Drama
b=[]
for genres in df["Converted_genres"]:
    if "Drama" in genres:
        b.append(1)
    else:
        b.append(0)
df["Drama"]=b
#add dummy for Thriller
c=[]
for genres in df["Converted_genres"]:
    if "Thriller" in genres:
        c.append(1)
    else:
        c.append(0)
df["Thriller"]=c
#add dummy for Family
d=[]
for genres in df["Converted_genres"]:
    if "Family" in genres:
        d.append(1)
    else:
        d.append(0)
df["Family"]=d
#add dummy for Romance
e=[]
for genres in df["Converted_genres"]:
    if "Romance" in genres:
        e.append(1)
    else:
        e.append(0)
df["Romance"]=e
#add dummy for Action
f=[]
for genres in df["Converted_genres"]:
    if "Action" in genres:
        f.append(1)
    else:
        f.append(0)
df["Action"]=f
#add dummy for Science Fiction
g=[]
for genres in df["Converted_genres"]:
    if "Science Fiction" in genres:
        g.append(1)
    else:
        g.append(0)
df["Science Fiction"]=g
#add dummy for Crime
h=[]
for genres in df["Converted_genres"]:
    if "Crime" in genres:
        h.append(1)
    else:
        h.append(0)
df["Crime"]=h
#add dummy for Adventure
i=[]
for genres in df["Converted_genres"]:
    if "Adventure" in genres:
        i.append(1)
    else:
        i.append(0)
df["Adventure"]=i
#add dummy for Adventure
j=[]
for genres in df["Converted_genres"]:
    if "Mystery" in genres:
        j.append(1)
    else:
        j.append(0)
df["Mystery"]=j



In [15]:
df.head(30)

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,Comedy,Drama,Thriller,Family,Romance,Action,Science Fiction,Crime,Adventure,Mystery
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,...,1,0,0,0,0,0,0,0,0,0
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,...,1,1,0,1,1,0,0,0,0,0
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,...,0,1,0,0,0,0,0,0,0,0
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,...,0,1,1,0,0,0,0,0,0,0
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,...,0,0,1,0,0,1,0,0,0,0
5,6,NaN,8000000,"[{'id': 16, 'name': 'Animation'}, {'id': 12, '...",NaN,tt0093743,en,Pinocchio and the Emperor of the Night,"Pinocchio and his friends, a glow worm and a m...",0.743274,...,0,0,0,1,0,0,0,0,1,0
6,7,NaN,14000000,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",http://www.thepossessionmovie.com/,tt0431021,en,The Possession,A young girl buys an antique box at a yard sal...,7.286477,...,0,0,1,0,0,0,0,0,0,0
7,8,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,tt0391024,en,Control Room,A chronicle which provides a rare window into ...,1.949044,...,0,0,0,0,0,0,0,0,0,0
8,9,"[{'id': 256377, 'name': 'The Muppet Collection...",0,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",NaN,tt0117110,en,Muppet Treasure Island,After telling the story of Flint's last journe...,6.902423,...,1,0,0,1,0,1,0,0,1,0
9,10,NaN,6000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",NaN,tt0310281,en,A Mighty Wind,"In ""A Mighty Wind"", director Christopher Guest...",4.672036,...,1,0,0,0,0,0,0,0,0,0


In [16]:
# space for sick scatter plots and vivacious violin plots
y1=y_train
y2=(y_train)**(1/8)
y3=np.log(y_train)

## 2. Feature Engineering and Data Cleaning

Transform your data into a cleaned DataFrame with the features you believe will be the most helpful towards creating a model for the revenue from the film.

In order to use the models below, you will need to make every feature **numerical**, not categorical, so you need to make sure that your output DataFrame only has numbers in it (and no NaNs!).

Some of the columns have data that is a little funky, so here's the libraries I imported and a few functions that I used. Feel free to use them or not!

In [17]:
from sklearn.decomposition import PCA
from collections import defaultdict
from sklearn.preprocessing import StandardScaler

def empty_listify(lst):
    return [] if pd.isnull(lst) else eval(lst)


def pcaify(one_hot, column_prefix, num_pca_columns):
    pca = PCA(n_components=num_pca_columns)    
    features = pca.fit_transform(one_hot)
    
    return pd.DataFrame(data = features, columns = ['{0}_{1}'.format(column_prefix, i) for i in range(features.shape[1])])

In [18]:
def feature_engineering(df):
    # change this with your own feature engineering!
    df=df.loc[:, ["budget", "popularity", "runtime", "belong", "language","Comedy","Drama","Thriller","Romance","Action","Family","Crime","Science Fiction","Adventure","Mystery"]]
    df=df.fillna(0)
    return df 

In [19]:
X = feature_engineering(df)

In [20]:

# Splitting up our cleaned df back into training and test
X_train = X[:train.shape[0]]
y_train = y2
X_test = X[train.shape[0]:]
X_train

,budget,popularity,runtime,belong,language,Comedy,Drama,Thriller,Romance,Action,Family,Crime,Science Fiction,Adventure,Mystery
0,14000000,6.575393,93.0,0,1,1,0,0,0,0,0,0,0,0,0
1,40000000,8.248895,113.0,0,1,1,1,0,1,0,1,0,0,0,0
2,3300000,64.299990,105.0,1,1,0,1,0,0,0,0,0,0,0,0
3,1200000,3.174936,122.0,1,0,0,1,1,0,0,0,0,0,0,0
4,0,1.148070,118.0,1,0,0,0,1,0,1,0,0,0,0,0
5,8000000,0.743274,83.0,1,1,0,0,0,0,0,1,0,0,1,0
6,14000000,7.286477,92.0,1,1,0,0,1,0,0,0,0,0,0,0
7,0,1.949044,84.0,1,1,0,0,0,0,0,0,0,0,0,0
8,0,6.902423,100.0,0,1,1,0,0,0,1,1,0,0,1,0
9,6000000,4.672036,91.0,1,1,1,0,0,0,0,0,0,0,0,0


<span id="modeling"/>

## 3. Modeling

For each of the models we try, make sure you also run the [Prediction](#prediction) cells at the bottom, so you can submit your predictions to the competition! This is how we'll be making sure you're keeping up with the project.

<span id="validation"/>

### 3.1 Validation and Evaluation

Our Kaggle competition (read more [here](https://www.kaggle.com/c/tmdb-box-office-prediction/overview/evaluation) uses Root-Mean-Square-Log-Error (RMSLE). In mathematical notation, it is:

$$\text{RMSLE}(\hat{y}, y) = \sqrt{\frac{1}{n} \sum_{i = 1}^n \log(y_i - \hat{y}_i)}$$

#### Evaluation

Complete the function below.

In [21]:

from sklearn.metrics import mean_squared_log_error

def evaluate(y_pred, y_true):
    """Returns the RMSLE(y_pred, y_true)"""
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

In [22]:
# Tests the previous function

# If this fails, it will throw an error
assert np.allclose(evaluate(np.array([1, 2, 3, 4]), np.array([5, 6, 7, 8])), 0.8292781201720374)

#### Validation

Use the [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to split up your training data into a training set and a validation set. The size of the validation set should be 20% of the full training data.

In [23]:
from sklearn.model_selection import train_test_split

train_X, valid_X, train_y, valid_y = train_test_split(X_train, y_train)


<span id="linear-regression"/>

### 3.2 Linear Regression

Fit a linear regression model to your data and report your RMLSE.

In [24]:
from sklearn.linear_model import LinearRegression

In [25]:
# instantiating linear regression object (model)
lm = LinearRegression()

# fitting model on training sets
lm.fit(train_X, train_y)

# using model to predict on validation set
y_valid_pred = (lm.predict(valid_X))**8

# IMPORTANT: This model is a "dumb" model that predicts negative values for some movie revenues
# However, because we are using RMLSE we cannot have negative predictions
# Ideally you create a better model that doesn't predict negative revenues
y_valid_pred[y_valid_pred < 0] = 0

# evaluating prediction on validation set
evaluate(y_valid_pred, valid_y)


14.17548920726308

<span id="reg" />

### 3.3 Regularized Regression

Fit a [LASSO regression model](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) to your data with $\lambda = 1$

In [26]:
from sklearn.linear_model import Lasso

In [27]:
Lasso_regression=Lasso(alpha=10)
Lasso_regression.fit(train_X, train_y)
Lasso_valid_predict=(Lasso_regression.predict(valid_X))**8
Lasso_valid_predict[Lasso_valid_predict<0]=0
evaluate(Lasso_valid_predict, valid_y)

14.188936790680449

In [28]:
#Decision Tree Regression
dt_reg = DecisionTreeRegressor(max_depth=5)
dt_reg.fit(train_X, train_y)

# Predict
dt_y_pred = dt_reg.predict(valid_X)**8
dt_y_pred[dt_y_pred<0]=0
evaluate(dt_y_pred, valid_y)

14.067844133074558

#### 3.3.1 Hyperparameter Tuning

Perform [3-fold cross-validation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) on the parameter $\lambda$, which is called **alpha** when you pass it into Lasso. Find the best parameter of $\lambda \in \{0.001, 0.005, 0.01, 0.05, 0.1\}$ and report the **RMSLE** on the validation set if you use this parameter.

In [29]:
from sklearn.model_selection import GridSearchCV
LS=GridSearchCV(Lasso(), {
               'alpha':[1e-3, 5e-3, 1e-2, 5e-2, 0.1,10,0.03,100]},
                cv=5,
                return_train_score=False)
LS.fit(train_X, train_y)
LS.cv_results_
CVresults=pd.DataFrame(LS.cv_results_)
CVresults[["param_alpha","mean_test_score"]]

,param_alpha,mean_test_score
0,0.001,0.448833
1,0.005,0.448685
2,0.01,0.447615
3,0.05,0.434393
4,0.1,0.411896
5,10,0.362291
6,0.03,0.441024
7,100,0.362291


In [30]:
best_alpha = alphas[np.argmax(cv_scores)]
best_alpha

NameError: name 'alphas' is not defined

In [31]:
Lasso_best_model = Lasso(alpha=best_alpha)
Lasso_best_model.fit(train_X, train_y)
Lasso_best_model_pred=Lasso_best_model.predict(valid_X)**8
evaluate(Lasso_best_model_pred, valid_y)

NameError: name 'best_alpha' is not defined

<span id="random-forest"/>

### 3.4 Random Forest

Fit a random forest model to your data and report your RMSLE.

**NOTE:** If you're finding that your model is performing worse than your linear regression, make sure you tune the parameters to the RandomForestRegressor!

Try to understand what the parameters mean by looking at the Decision Trees lecture.

In [32]:
from sklearn.ensemble import RandomForestRegressor

In [33]:
#Random Forest Regression
rf_reg = RandomForestRegressor(n_estimators=60, random_state=0)
rf_reg.fit(train_X, train_y.values.ravel())

# Predict
rf_y_pred = (rf_reg.predict(valid_X))**8
rf_y_pred[rf_y_pred<0]=0
evaluate(rf_y_pred, valid_y)

14.079720304383638

In [34]:
#Hypterparameter tuning for Random forest
RF=GridSearchCV(RandomForestRegressor(),
               {"n_estimators":[10,20,30,40,50,60],
               }, 
                cv=5,
               return_train_score=False)
RF.fit(train_X, train_y)
RFresults=pd.DataFrame(RF.cv_results_)


In [35]:
RFresults[["param_n_estimators","mean_test_score"]]
RF.best_params_

{'n_estimators': 60}



<span id="xgb" />

### 3.6 XGBoost (Stretch)

Now that we've tried many different types of classifiers, it's time to bring out the big guns.

Below are hyperparameters for an XGBoost model: tinker around with these to achieve the best validation score (below). Learn about what some of the hyperparameters mean [here](https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.train).

**NOTE**: You will probably run into issues to run this on DataHub! I would recommend downloading Anaconda and running the notebook locally. Ask us on Slack if you need help on this!

In [100]:
from xgboost import train

In [109]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)
xg_reg.fit(train_X,train_y)
pred=xg_reg.predict(valid_X)**8
evaluate(pred, valid_y)



11.090578661821514

## Prediction

In [152]:
PATH_TO_SUBMISSION = 'submission.csv'

In [153]:
# You might have to change this to be the predictions from your model on the test set
preds = (rf_reg.predict(X_test))**8

In [154]:
out = pd.DataFrame(data={'id': test['id'], 'revenue': preds}).set_index('id')

In [155]:
assert out.shape[0] == test.shape[0]

In [156]:
out.to_csv(PATH_TO_SUBMISSION)

## Submission

In [157]:
from submit import submit_to_leaderboard

In [ ]:
success = submit_to_leaderboard(
    KAGGLE_USER_DATA, 
    TEAM_NAME, 
    path_to_submission=PATH_TO_SUBMISSION, 
    submit_to_kaggle=True
)

Submitting to Kaggle...
Enter a commit message for your submission: commit
Reading Kaggle submissions...
